In [1]:
import skopt
from skopt.space import Real, Integer
import numpy as np
import pandas as pd
import sliceoptim.gcode as gc
import sliceoptim.core as core

In [2]:
printer = core.Printer(
            bed_size=[220, 220],
            nozzle_diameter=0.4,
            max_speed=120,
            min_speed=5
        )
filament = core.Filament(
            kind="pla",
            extrusion_temp_range=[180, 250],
            bed_temp_range=[25, 80],
            diameter=1.75,
        )

In [4]:
opt = skopt.Optimizer(space, "GP", acq_func="PI",
                      acq_optimizer="auto",
                      initial_point_generator="lhs",
                      n_initial_points=40)

In [5]:
samples = pd.read_csv('first_layer_experiments_graded_2.csv').drop(columns='Unnamed: 0').reset_index(drop=True)
grades = pd.read_csv('first_layer_grades_2.csv').drop(columns='sample').reset_index(drop=True)

In [6]:
samples = samples.drop(columns=['temperature', 'bed-temperature'])

In [7]:
for s in samples.iterrows():
    opt.tell(list(s[1]), grades.at[s[0], 'grade'])

In [8]:
new_acq_params = {"xi": 0.01}
opt.acq_func_kwargs = new_acq_params
opt.update_next()

In [9]:
opt.ask(20)

[[211, 50, 8, 0.39335169020768873, 0.576910708208415, 0.807496925920746],
 [204, 47, 41, 0.33566772228425457, 0.5403163957087957, 0.9364956388983213],
 [205, 48, 41, 0.339174368336293, 0.545569617079988, 0.9362712180115991],
 [194, 51, 6, 0.356512108298031, 0.5668469131939872, 0.8137741557391821],
 [190, 46, 20, 0.3927126871761246, 0.6369885543097433, 0.8057544562681702],
 [190, 50, 5, 0.3676009055856014, 0.36000000000000004, 0.8514139840801043],
 [190, 43, 44, 0.4, 0.4071705717544456, 0.9738536484757213],
 [190, 42, 38, 0.4, 0.36000000000000004, 0.9718067483397952],
 [199, 50, 6, 0.39857544246266907, 0.5718557303334648, 0.814417894546491],
 [190, 51, 5, 0.3254395981460219, 0.36000000000000004, 0.8518384823028554],
 [194, 50, 10, 0.305235610979456, 0.5941121822350386, 0.8146440071139689],
 [214, 47, 42, 0.31629571049761507, 0.5286234345759427, 0.940810032434759],
 [190, 42, 43, 0.4, 0.37871922132522606, 0.9756554977330265],
 [211, 48, 21, 0.4, 0.6341286579284677, 0.8086788078817906],
 

In [10]:

res = opt.get_result()

In [92]:
s = skopt.expected_minimum(res, 100)

In [112]:
a, b = res.models[0].predict([s[0]], return_std=True)

In [94]:
d = {"a": "x", "b": "y", "c": "z"}
for i, k in enumerate(d):
    print(str(i) + " : " + str(k))

0 : a
1 : b
2 : c


In [107]:
space[1][1].name

'first-layer-bed-temperature'

In [118]:
res.func_vals # permet de récuper un pourcentage pour CI

array([1. , 4. , 5. , 1. , 3. , 2. , 3. , 2. , 1. , 1. , 3. , 1. , 2. ,
       1. , 2. , 3. , 2. , 1. , 1. , 2.5, 1.5, 0.5, 2. , 2. , 2. , 2. ,
       2. , 0.5, 1. , 1. , 2. , 2. , 2. , 3. , 3. , 2. , 2. , 1. , 1. ,
       2. ])